In [2]:
#To import the relevant libraries
import pandas as pd
import numpy as np
import io
!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
#To read in the data
bigheart_apriori = pd.read_excel('BigHeart_GBA.xlsx', sheet_name='BigHeart')
display(bigheart_apriori)

,VolID,Age,Gender,MStat,AttendTrain,Rotation,PrevVolExp,Conflict,PeerSupport,SatRating,Feedback,NoShow,NumVol,Churn
0,1,37,Male,Single,No,No,No,No,Yes,7,2,9,18,No
1,2,30,Female,Single,No,No,Yes,Yes,Yes,4,0,6,4,No
2,3,43,Male,Married,Yes,Yes,Yes,No,Yes,7,2,9,12,No
3,4,31,Female,Single,Yes,Yes,Yes,Yes,Yes,1,0,5,16,No
4,5,24,Female,Single,No,Yes,No,Yes,Yes,4,0,7,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1496,28,Male,Single,No,Yes,Yes,Yes,No,5,4,5,2,No
1496,1497,42,Male,Single,Yes,No,No,No,No,6,1,7,4,No
1497,1498,28,Female,Single,No,No,Yes,Yes,No,5,0,8,12,No
1498,1499,35,Female,Married,Yes,Yes,No,No,No,4,4,3,8,Yes


In [4]:
#Check if there are any null values
bigheart_apriori.isnull().sum()


,0
VolID,0
Age,0
Gender,0
MStat,0
AttendTrain,0
Rotation,0
PrevVolExp,0
Conflict,0
PeerSupport,0
SatRating,0


In [5]:
#Check if there are any duplicate data
bigheart_apriori.duplicated(subset=['VolID'])

,0
0,False
1,False
2,False
3,False
4,False
...,...
1495,False
1496,False
1497,False
1498,False


In [6]:
# Data Prep [Part 1]
# Convert Gender & MStat to flag variables

# Gender conversion
bigheart_apriori['Male'] = bigheart_apriori['Gender'].apply(lambda x: True if x == 'Male' else False)
bigheart_apriori['Female'] = bigheart_apriori['Gender'].apply(lambda x: True if x == 'Female' else False)

# MStat conversion
bigheart_apriori['Married'] = bigheart_apriori['MStat'].apply(lambda x: True if x == 'Married' else False)
bigheart_apriori['Single'] = bigheart_apriori['MStat'].apply(lambda x: True if x == 'Single' else False)
bigheart_apriori['Others'] = bigheart_apriori['MStat'].apply(lambda x: True if x == 'Others' else False)


# Data Prep [Part 2]
# Convert columns with Yes/No values to True/False as Apriori algorithm only handles True/False OR 1/0 values
cols_to_convert = ['AttendTrain', 'Rotation', 'PrevVolExp', 'Conflict', 'PeerSupport', 'Churn']
for col in cols_to_convert:
      bigheart_apriori[col] = bigheart_apriori[col].replace({'Yes': True, 'No': False})

# Drop the original columns
bigheart_apriori = bigheart_apriori.drop(['Gender', 'MStat'], axis=1)

# Display dataframe
display(bigheart_apriori)

<ipython-input-6-da29355f5945>:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bigheart_apriori[col] = bigheart_apriori[col].replace({'Yes': True, 'No': False})


,VolID,Age,AttendTrain,Rotation,PrevVolExp,Conflict,PeerSupport,SatRating,Feedback,NoShow,NumVol,Churn,Male,Female,Married,Single,Others
0,1,37,False,False,False,False,True,7,2,9,18,False,True,False,False,True,False
1,2,30,False,False,True,True,True,4,0,6,4,False,False,True,False,True,False
2,3,43,True,True,True,False,True,7,2,9,12,False,True,False,True,False,False
3,4,31,True,True,True,True,True,1,0,5,16,False,False,True,False,True,False
4,5,24,False,True,False,True,True,4,0,7,4,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1496,28,False,True,True,True,False,5,4,5,2,False,True,False,False,True,False
1496,1497,42,True,False,False,False,False,6,1,7,4,False,True,False,False,True,False
1497,1498,28,False,False,True,True,False,5,0,8,12,False,False,True,False,True,False
1498,1499,35,True,True,False,False,False,4,4,3,8,True,False,True,True,False,False


In [7]:
#Examine the distributon of the variables
# Exclude columns with numeric values
columns_to_avoid = ['VolID', 'Age', 'SatRating', 'Feedback', 'NoShow', 'NumVol']
cols_to_analyze = [col for col in bigheart_apriori.columns if col not in columns_to_avoid]

# Create an empty dictionary to store results
results = {}

# Iterate through columns and calculate frequency and percentage for 'True' values
for col in cols_to_analyze:
  true_count = (bigheart_apriori[col] == True).sum()
  total_count = bigheart_apriori[col].count()
  true_percentage = (true_count / total_count) * 100 if total_count > 0 else 0

  results[col] = {'Frequency (True)': true_count, 'Percentage (True)': true_percentage}

# Create a new DataFrame from the results dictionary
comparison_df = pd.DataFrame.from_dict(results, orient='index')

# Display the comparison chart
print(comparison_df)

             Frequency (True)  Percentage (True)
AttendTrain               838          55.866667
Rotation                  597          39.800000
PrevVolExp                606          40.400000
Conflict                  561          37.400000
PeerSupport               624          41.600000
Churn                     502          33.466667
Male                      634          42.266667
Female                    866          57.733333
Married                   661          44.066667
Single                    772          51.466667
Others                     67           4.466667


In [9]:
# Create a new DataFrame without the numeric column
bigheart_analysis = bigheart_apriori.copy()
bigheart_analysis = bigheart_analysis.drop(['VolID', 'Age', 'SatRating', 'Feedback', 'NoShow', 'NumVol'], axis=1)

#tabulate the frequent itemsets with confidence % more than 60% [Rule Support = 0.60 * 0.10(Antecedent Support) = 0.06]
frequent_itemsets = apriori(bigheart_analysis, min_support=0.06, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.60, num_itemsets=5)

#Select rules with consequent as Churn
rules = rules[rules['consequents'].apply(lambda x: 'Churn' in x)]
rules_with_churn = rules[rules['consequents'] == {'Churn'}]

#Sort the rules by descending confidence
rules_sorted = rules_with_churn.sort_values(by='confidence', ascending=False)

#Select rules with antecedent support more than 10%
rules_filtered=rules_sorted[rules_sorted['antecedent support'] >= 0.10]

display(rules_filtered)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
141,"(Married, Conflict, Female)",(Churn),0.111333,0.334667,0.079333,0.712575,2.129208,1.0,0.042074,2.314806,0.596784,0.216364,0.567998,0.474813
159,"(Male, PrevVolExp, Single, Rotation)",(Churn),0.111333,0.334667,0.074000,0.664671,1.986068,1.0,0.036740,1.984119,0.558694,0.198925,0.495998,0.442893
135,"(Male, PrevVolExp, Single)",(Churn),0.122000,0.334667,0.080000,0.655738,1.959376,1.0,0.039171,1.932635,0.557669,0.212389,0.482572,0.447391
75,"(Rotation, AttendTrain, Female)",(Churn),0.110667,0.334667,0.072000,0.650602,1.944031,1.0,0.034964,1.904230,0.546033,0.192857,0.474853,0.432871
63,"(Married, Conflict)",(Churn),0.134000,0.334667,0.084000,0.626866,1.873105,1.0,0.039155,1.783093,0.538253,0.218371,0.439177,0.438931
104,"(Rotation, Male, PrevVolExp)",(Churn),0.153333,0.334667,0.094000,0.613043,1.831803,1.0,0.042684,1.719401,0.536327,0.238579,0.418402,0.446960


2 overlapping rules:
Rules [2 & 4] and [4 & 6] are overlapping as they have a common feature of Rotation


2 subset rules:

*   Rule 1 is a subset of Rule 5

*   Rule 2 is a subset of Rule 3 and Rule 6


3 rule sets:
*  Rule Set 1: Rules 1 & 5
*  Rule Set 2: Rules 2,3,6
*  Rule Set 3: Rule 4



Discuss:


*   For Rule Set 1: For volunteers who are female, married and had faced any conflict at work and/or home in the past three months, counselling services can be provided or provide them with leave of absence to work on their issues before returning to work.

*   For Rule Set 2: For volunteers who are male, single, have previous volunteering experience before joining BigHeart and rotated to other activities in the past three months, it could be that BigHeart is not up to standards than their previous work experiences, so BigHeart could interview them and get suggestions on what can be improved.


*   For Rule Set 3: For volunteers who are female, attended training, rotated to other volunteering events, training can be further improved to cater to individuals who do not have previous volunteering experience to provide them encouragement to take up volunteering




